<a href="https://colab.research.google.com/github/cc-ca/commentaires_toxiques/blob/C%C3%A9lestin/Classification_commentaires_toxiques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importation des packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Importation des données

Ajoutez un raccourci de ce dossier à votre google drive :

https://drive.google.com/drive/folders/1mx-CAzT10YKrmxHfYDP_1Oef7PVGUr7s?usp=sharing

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/train.csv')
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


# Etude du jeu de données

In [5]:
toxic=data.loc[:,"toxic"]
severe_toxic=data.loc[:,"severe_toxic"]
obscene=data.loc[:,"obscene"]
threat=data.loc[:,"threat"]
insult=data.loc[:,"insult"]
identity_hate=data.loc[:,"identity_hate"]


def compter(colone):
  cpt=0
  for x in colone:
    if x==1:
      cpt=cpt+1
  return cpt

# on remarque que certains commmentaires ont plusieurs tags toxiques
print('toxic : '+ str(compter(toxic)))
print('severe_toxic : '+ str(compter(severe_toxic)))
print('obscene : '+ str(compter(obscene)))
print('threat : '+ str(compter(threat)))
print('insult : '+ str(compter(insult)))
print('identity_hate : '+ str(compter(identity_hate)))

toxic : 15294
severe_toxic : 1595
obscene : 8449
threat : 478
insult : 7877
identity_hate : 1405


In [6]:
def compter_sanstag(colone1,colone2,colone3,colone4,colone5,colone6):
  cpt=0
  for x in range (len(colone1)):
    if colone1[x]==0 and colone2[x]==0 and colone3[x]==0 and colone4[x]==0 and colone5[x]==0 and colone6[x]==0:
      cpt=cpt+1
  return cpt


print("commentaires sans haine : "+ str(compter_sanstag(toxic,severe_toxic,obscene,threat,insult,identity_hate)))

commentaires sans haine : 143346


In [7]:
# Your Code
print(type(data))


<class 'pandas.core.frame.DataFrame'>


# Préparation des données

In [8]:
data=data.drop(columns=['id'])

0    0
1    0
2    0
3    0
4    0
Name: toxic, dtype: int64

# Entraînement du modèle baseline

In [11]:
# Your Code

# Itération de la modélisation 

In [12]:
# Your Code 